Dado que ya contamos con la clase encargada de conectarse a MLflow y obtener los modelos en producción, así como con la clase que supervisa los datos de entrada, es momento de construir la API.

1. Se importan las librerías básicas necesarias para FastAPI.

2. Se incorporan las clases previamente creadas: la encargada de la conexión con MLflow y aquella que define la estructura de los datos que el modelo requiere para realizar predicciones.

3. Se configura el sistema de registros (logging), que permitirá monitorear el comportamiento de la aplicación durante su ejecución (alertas, errores, entre otros). Asimismo, se definen las variables globales que funcionarán como memoria caché: un diccionario de modelos (ml_models) y un diccionario de manejadores de servicios (service_handlers). Y se lee el archivo con las credenciales de minio para poder acceder a los modelos.

4. Se establece el contexto de ciclo de vida (lifespan) para la aplicación FastAPI, lo que permite controlar las acciones que ocurren antes de que la aplicación comience a atender peticiones y después de que se apague. Para ello, se utiliza el decorador @asynccontextmanager, cuyo propósito es definir un bloque que gestiona recursos que requieren inicialización y liberación (setup / teardown) dentro de funciones asíncronas. La primera función async es lifespan(app: FastAPI), en la cual, al iniciar la aplicación, se ejecuta el siguiente servicio:

    -  `service_handlers['mlflow'] = MLFlowHandler()`

    Este servicio crea una única instancia de MLFlowHandler y la almacena en el diccionario global service_handlers, de modo que cualquier parte del código (por ejemplo, los endpoints de FastAPI) pueda acceder a ella sin necesidad de volver a inicializarla. Además, se registra en el log que el manejador ha sido inicializado, y se hace mediante la siguiente línea:

    - `logging.info("Initialised mlflow handler {}".format(type(service_handlers['mlflow'])))`

    Posteriormente, tras el yield, cuando la aplicación se detiene, se limpian los diccionarios **service_handlers** y **ml_models** para liberar memoria y cerrar conexiones. Finalmente, se deja constancia en el log de que todo ha sido limpiado.Todo este proceso se hace con las siguientes líneas:

    - yield
    - service_handlers.clear()
    - ml_models.clear()
    - logging.info("Handlers y ml models limpiado")

5. Se crea la aplicación con FastAPI, en donde se le establece los siguientes parámetros:
    - título: API de Evaluación de Riesgo Crediticio.
    - descripción: Servicio que estima la probabilidad de incumplimiento de un préstamo dentro de los 12 meses posteriores a su adquisición.

    Este aplicación se conecta al ciclo de vida de la aplicación mediante el código la linea:
    - lifespan=lifespan

6. Se define un `endpoint HTTP GET` en la aplicación FastAPI, disponible en la ruta:

    - `http://<tu-servidor>/health/:` El propósito de este endpoint es reportar el estado de salud del sistema de machine learning, principalmente la conexión con MLflow y el estado del Model Registry.

    Cuando una aplicación cliente (por ejemplo, postman) envía una solicitud a /health/, FastAPI ejecuta la función `async def healthcheck()`. Esa función hace tres llamadas importantes (todas a través del objeto service_handlers['mlflow'])

    - `"serviceStatus: "OK""`  Simplemente indica que el servidor FastAPI está funcionando y ha podido procesar la petición.

    - `"modelTrackingHealth": service_handlers['mlflow'].check_mlflow_health()`  Comprueba si el servidor MLflow está disponible y responde correctamente. Si todo esta bien devuelve **"Service returning experiments"**.

    - `"modelRegistryHealth": service_handlers['mlflow'].check_registry_health()`  Consulta el registro de modelos en MLflow para comprobar si existe al menos una versión de modelo en el estado de **"Production"**. Devuelve True si al menos un modelo está en producción.

    - `"productionModels": service_handlers['mlflow'].list_production_models()`  Lista todos los modelos actualmente en etapa **“Production”** en el registro. Devuelve una lista de diccionarios con nombre del modelo, versión , descipción, etc.


7. Se define un `endpoint HTTP GET` en la aplicación FastAPI, disponible en la ruta:

    - `http://<tu-servidor>/debug/mlflow/:` El propósito de este endpoit es que cada vez que se haga una petición GET a esta url, FastAPI ejecuta la función **async def debug_mlflow()** y se llama al método `debug_registry()` mediante la linea *service_handlers['mlflow'].debug_registry()*. Este se encarga de obtener la información completa del estado actual del Model Registry de MLflow. Devolviendo así todos los modelos registrados y sus versiones más recientes incluyendo su nombre, descripción, estado, etapa (stage) y versiones que están actualmente en producción.

8. Se define un `endpoint HTTP POST` el cual recibe los datos de entrada y detecta si la entrada proviene de un archivo CSV subido (UploadFile) o
un cuerpo JSON, los transforma en un DataFrame válido mediante `ClassificationRequest`, y utiliza un modelo de `scikit-learn` cargado desde caché para generar predicciones de probabilidad por clase. Si el modelo no se encuentra en caché, se lanza un error HTTP 500.  En caso de error en el procesamiento de los datos o la inferencia, se devuelve un error HTTP 400.

In [ ]:
# librerías generales
import os
os.chdir("/media/luisgarcia/Datos/32. Ejercicio de acercamiento al rol/mlops_pyme/src/serve")

# =============================================
# 1. IMPORTACIONES GENERALES (lo básico de FastAPI)
# =============================================
from fastapi import FastAPI,HTTPException,UploadFile,File,Request
from contextlib import asynccontextmanager # General: para manejar ciclo de vida
import logging                             # General: para obtener los log de la aplicación
import time
import json
from pathlib import Path
import os

# =============================================
# 2. Importar los ódigos creados para comunicarnos con MLflow y validar datos  de entrada
# =============================================
# Usemos las dos clases que creamos MlflowHandler y ClassificationRequest
from helpers.schemas import ClassificationRequest
from registry.mlflow.mlflow_handler import MlflowHandler

# =============================================
# 3. CONFIGURACIÓN INICIAL
# =============================================
# Crear un sistema de registro que permite rastrear lo que sucede en la aplicación mientras se ejecuta.
log_format = "%(asctime)s - %(name)s - %(levelname)s - %(message)s" 
logging.basicConfig(format = log_format, level = logging.INFO)

# configuremos las variables globales (caché)
ml_models = {}
service_handlers = {}

# carga credenciales de minio
project_secrets = next(p for p in Path.cwd().parents if (p / 'data').exists())
credential_path = lambda file: os.path.join(project_secrets,'secrets',file)
credential_path = credential_path('credentials_minio.json')
with open(credential_path, 'r') as file:
    credentials_minio = json.load(file)
# =============================================
# 4. CICLO DE VIDA DE LA APP
# =============================================
@asynccontextmanager
async def lifespan(app: FastAPI): # primera función función que al iniciar la aplicación ejecuta los siguientes servicios
    """
    Ciclo de vida de la aplicación FastAPI.

    Inicializa los servicios necesarios al arrancar la aplicación, 
    incluyendo la conexión con el servidor MLflow y la carga del modelo 
    de producción en memoria caché.  
    Al detener la aplicación, libera los recursos y limpia los servicios 
    registrados.

    Parameters
    ----------
    app : FastAPI
        Instancia principal de la aplicación FastAPI.

    Process
    --------
    - Crea una instancia de `MlflowHandler` configurada con las credenciales de MinIO.
    - Carga el modelo `DecisionTree_CreditRiskModel` desde la etapa *Production* 
      de MLflow y lo almacena en caché.
    - Registra los eventos relevantes en el sistema de logs.
    - Tras finalizar la ejecución de la aplicación, limpia las variables 
      globales `service_handlers` y `ml_models`.

    Yields
    ------
    None
        Control temporal devuelto al manejador de contexto de FastAPI 
        durante la ejecución de la aplicación.

    Raises
    -----------
    Exception
        Si ocurre un error al intentar cargar el modelo desde MLflow 
        durante la inicialización.
    """
    service_handlers['mlflow'] = MlflowHandler(
        tracking_uri="http://localhost:5000",
        s3_endpoint=credentials_minio["endpoint_url"],
        aws_access_key=credentials_minio["aws_access_key_id"],
        aws_secret_key=credentials_minio["aws_secret_access_key"]
)
    logging.info("Inicializado MlflowHandler {}".format(type(service_handlers['mlflow'])))
    # cargar el modelo
    model_name = "DecisionTree_CreditRiskModel"
    try:
        ml_models[model_name] = service_handlers['mlflow'].get_production_sklearn_model(model_name)
        logging.info(f"Modelo '{model_name}' cargado y almacenado en caché.")
    except Exception as e:
        logging.error(f"No se pudo cargar el modelo '{model_name}' al iniciar: {e}")
    
    yield
    # lo que se ejecuta después de cerrar la aplicación
    service_handlers.clear()
    ml_models.clear()
    logging.info("Servicios Handlers y ml models limpiados")

# =============================================
# 5. CREAR LA APLICACIÓN FASTAPI
# =============================================
app = FastAPI(
    title="API de Evaluación de Riesgo Crediticio",
    description="Servicio que estima la probabilidad de incumplimiento " \
    "de un préstamo dentro de los 12 meses" \
    "posteriores a su adquisición.",
    lifespan=lifespan  # Conectar el ciclo de vida
)

# =============================================
# 6. DEFINIR EL ENDPOINT HTTP GET PARA COMPROBAR ESTADO DEL SERVIDOR MLFLOW
# =============================================
@app.get("/health/", status_code=200)
async def healthcheck():
    """
    Verifica el estado general del servicio y del servidor de MLflow.

    Este endpoint permite comprobar que la API está activa y que los 
    servicios asociados a MLflow (tanto el servidor de tracking como 
    el registro de modelos) están funcionando correctamente. 
    Además, devuelve la lista de modelos actualmente en la etapa 
    *Production* dentro del registro de MLflow.

    Returns:
        dict: Un diccionario con información sobre el estado de la API 
        y del entorno de MLflow, incluyendo:
            - **serviceStatus** (`str`): Estado general del servicio (`"OK"` si está activo).
            - **modelTrackingHealth** (`bool`): Estado del servidor de tracking de MLflow.
            - **modelRegistryHealth** (`bool`): Estado del registro de modelos.
            - **productionModels** (`list[str]`): Lista de modelos en la etapa *Production*.
    """
    return {
        "serviceStatus": "OK",
        "modelTrackingHealth": service_handlers['mlflow'].check_mlflow_health(),
        "modelRegistryHealth": service_handlers['mlflow'].check_registry_health(),
        "productionModels": service_handlers['mlflow'].list_production_models()
    }
# =============================================
# 7. DEFINIR EL ENDPOINT HTTP GET PARA OBTENER INFORMACIÓN COMPLETA DEL SERVIDOR MLFLOW
# =============================================
@app.get("/debug/mlflow/", status_code=200)
async def debug_mlflow():
    return service_handlers['mlflow'].debug_registry()

# =============================================
# 8. DEFINIR EL ENDPOINT HTTP POST PARA OBTENER LAS PREDICCIONES
# =============================================
@app.post("/classify/", status_code=200)
async def classify(file: UploadFile | None = File(None), request: Request = None):
    """
    Endpoint de clasificación de riesgo crediticio.

    Este endpoint recibe datos de entrada en formato CSV o JSON, los transforma en un 
    DataFrame válido mediante `ClassificationRequest`, y utiliza un modelo de 
    `scikit-learn` cargado desde caché para generar predicciones de probabilidad 
    por clase. 

    Si el modelo no se encuentra en caché, se lanza un error HTTP 500.  
    En caso de error en el procesamiento de los datos o la inferencia, 
    se devuelve un error HTTP 400.

    Parameters
    ----------
    file : UploadFile | None, opcional
        Archivo CSV cargado por el usuario con las observaciones a clasificar.
    request : Request, opcional
        Objeto de solicitud HTTP, usado para detectar si el contenido proviene de un
        archivo o de un cuerpo JSON.

    Return
    -------
    dict
        Un diccionario con la siguiente información:
        - **n_observaciones**: número de registros clasificados.
        - **clases**: lista con los nombres de las clases del modelo.
        - **predicciones**: lista de diccionarios con las probabilidades por clase.
        - **datos**: representación de los datos de entrada como diccionario de listas.
        - **tiempo_inferencia_seg**: tiempo total de inferencia en segundos.

    Raises
    -----------
    HTTPException
        - 400: Si ocurre un error al procesar los datos o realizar la clasificación.
        - 500: Si el modelo no se encuentra disponible en caché.
    """
    try:
        content_type = request.headers.get("content-type", "")
        if file:
            if file.filename.endswith(".csv"):
                content = await file.read()
                decoded = content.decode("utf-8")
                df = ClassificationRequest.from_input(decoded)
        
        elif "application/json" in request.headers.get("content-type", ""):
            data = await request.json()
            df = ClassificationRequest.from_input(data)

        else:
            return {"error": "Formato no reconocido"}
        
        
        model_name = "DecisionTree_CreditRiskModel"
        if model_name not in ml_models:
            raise HTTPException(status_code=500, detail="Modelo no encontrado en caché")
        model = ml_models[model_name]

        start = time.time()
        probabilities = model.predict_proba(df)

        classes = getattr(model, "clase_", [f"Clase_{i}" for i in range(probabilities.shape[1])])
        results = [
                {cls: float(prob) for cls, prob in zip(classes, row)}
                for row in probabilities
                ]
        inference_time = round(time.time() - start, 4)
        return {
                "n_observaciones": len(df),
                "clases": list(classes),
                "predicciones": results,
                "datos": df.to_dict(orient="list"),
                "tiempo_inferencia_seg": inference_time
            }
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error en la clasificación: {e}")